# Data: Email Marketing Campaign 

### Dataset :
- 33750 Custimers Id and name 
- date sinse January 2023 until end of December 2024
- email type: Welcome, Promotion, New Product, Cart Reminder, Reactivation
- email interractions: sent, opened, clicked, purchased



### Import libraries

In [1]:
import random 
import numpy as np 
import pandas as pd 
from faker import Faker 
from datetime import datetime, timedelta

### Seed and setup

In [2]:
random.seed(1009) 
np.random.seed(1009) 
Faker.seed(1009) 
faker = Faker('en_US') # set locale to US

In [3]:
N = 33750 # number of records 

email_types = { 
                1: "Welcome", 
                2: "Promotion", 
                3: "New Product", 
                4: "Cart Reminder", 
                5: "Reactivation" 
                }

### Probabilities by email type and Seasonal coefficients by month

In [4]:
#probability of email open
open_prob = {                
    1: 0.70, # Welcome 
    2: 0.60, # Promotion 
    3: 0.45, # New Product 
    4: 0.55, # Cart Reminder 
    5: 0.45  # Reactivation 
} 

#probability of clicking in the email
click_prob = { 
    1: 0.25, 
    2: 0.15,
    3: 0.12, 
    4: 0.35, 
    5: 0.08 } 

#probability of purchase conversion
purchase_prob = { 
    1: 0.07, 
    2: 0.05, 
    3: 0.02, 
    4: 0.15, 
    5: 0.06 
}

# Seasonal coefficients by month
seasonal_multiplier = {
    1: 1.05,  # Jan (promotions)
    2: 1.00,  
    3: 0.95,  
    4: 0.90,
    5: 1.00,  
    6: 1.10,  # June (promotions)
    7: 1.05,  
    8: 0.85,
    9: 0.95,  
    10: 1.00, 
    11: 1.15, # November (Black Friday)
    12: 1.25  # December (Holiday promo)
} 


# Generate data

In [5]:
def generate_customer_data(num_customers=33750, num_emails=5): # num_emails - max emails per person
    emails_compaigh = [] # List to store generated  data
    

    for i in range(N): 
        customer_id = faker.random_number(digits=7)
        name = faker.name() 
        
        # Define time range for email interactions
        start_time = datetime(year=2023, month=1,day=1)
        end_time = datetime(year=2024, month=12,day=31)
        
        # Generate a random email interaction time
        email_datetime = faker.date_time_between_dates(datetime_start=start_time, datetime_end=end_time)
        
        # Individual "noise" for the client
        personal_noise = np.random.normal(1.0, 0.15)
      
        
        for j in range(1, num_emails + 1): 
            sent_datetime = email_datetime + timedelta(days=j - 1)
            
            
            if start_time <= sent_datetime <= end_time:
                bounce_datetime = None
                open_datetime = None
                click_datetime = None
                transaction_datetime = None
                transaction_amount = None
                
                #Define bounce probability
                bounce_probability = random.random()
                if bounce_probability <= 0.02:
                    bounce_datetime = sent_datetime + timedelta(minutes=random.randint(0, 1))
                else:
                    # Get month and trand
                    month = sent_datetime.month
                    
                    # Add trend (growth open_rate)
                    days_from_start = (sent_datetime - start_time).days
                    trend_factor = 1 + (days_from_start / 730) * 0.10  # grow by 10% over 2 years
                    
                    # Сombining effects
                    open_adj = open_prob[j] * seasonal_multiplier[month] * trend_factor * personal_noise
                    click_adj = click_prob[j] * seasonal_multiplier[month] * trend_factor * personal_noise
                    purchase_adj = purchase_prob[j] * seasonal_multiplier[month] * trend_factor * personal_noise
                    
                    # Limit the range of probabilities to 0–1
                    open_adj = min(max(open_adj, 0), 1)
                    click_adj = min(max(click_adj, 0), 1)
                    purchase_adj = min(max(purchase_adj, 0), 1)
                    
                    if random.random() <= open_adj:
                        open_datetime = sent_datetime + timedelta(minutes=random.randint(1, 59))
                        if random.random() <= click_adj:
                            click_datetime = open_datetime + timedelta(minutes=random.randint(1, 59))
                            if random.random() <= purchase_adj:
                                
                                # Basic check, depends on the type of letter
                                base_min, base_max = {
                                        1: (500, 900),    # Welcome — smaller purchases
                                        2: (700, 1300),   # Promotion
                                        3: (800, 1500),   # New Product
                                        4: (1000, 2000),  # Cart Reminder — higher-value transactions
                                        5: (600, 1100)    # Reactivation
                                    } [j]
                                
                                # adjust by month, trend and personal noise
                                seasonal_factor = seasonal_multiplier[month]
                                amount_trend = 1 + (days_from_start / 730) * 0.05  # checks grow by 5% over 2 years
                                personal_spend_factor = np.random.normal(1.0, 0.1) # slight individual difference
                                
                                transaction_amount = round(random.uniform(base_min, base_max)* seasonal_factor * amount_trend * personal_spend_factor, 2)
                                transaction_datetime = click_datetime + timedelta(days=random.randint(1, 30)) # Random transaction date within 30 days after the click
                    
                
                
                # Append generated data to the list
                emails_compaigh.append({
                    "customer_id": customer_id,
                    "customer_name": name,
                    "email_type": email_types[j],
                    "sent_datetime": sent_datetime.strftime('%Y-%m-%d %H:%M:%S'),
                    "open_datetime": open_datetime.strftime('%Y-%m-%d %H:%M:%S') if open_datetime else None,
                    "click_datetime": click_datetime.strftime('%Y-%m-%d %H:%M:%S') if click_datetime else None,
                    "bounce_datetime": bounce_datetime.strftime('%Y-%m-%d %H:%M:%S') if bounce_datetime else None,
                    "purchase_datetime": transaction_datetime.strftime('%Y-%m-%d %H:%M:%S') if transaction_datetime else None,
                    "transaction_amount": transaction_amount
                })
    
    return emails_compaigh

### Run function

In [6]:
emails = generate_customer_data()

### Convert the list to a pandas DataFrame

In [7]:
data_emails = pd.DataFrame(emails)

In [8]:
data_emails

,customer_id,customer_name,email_type,sent_datetime,open_datetime,click_datetime,bounce_datetime,purchase_datetime,transaction_amount
0,3023127,Zachary Stevenson,Welcome,2023-12-16 06:39:16,2023-12-16 06:44:16,None,None,None,NaN
1,3023127,Zachary Stevenson,Promotion,2023-12-17 06:39:16,None,None,None,None,NaN
2,3023127,Zachary Stevenson,New Product,2023-12-18 06:39:16,None,None,None,None,NaN
3,3023127,Zachary Stevenson,Cart Reminder,2023-12-19 06:39:16,2023-12-19 06:57:16,2023-12-19 07:11:16,None,None,NaN
4,3023127,Zachary Stevenson,Reactivation,2023-12-20 06:39:16,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...
168221,6599109,Justin Brown,Welcome,2024-02-16 04:17:13,2024-02-16 05:11:13,None,None,None,NaN
168222,6599109,Justin Brown,Promotion,2024-02-17 04:17:13,2024-02-17 05:16:13,None,None,None,NaN
168223,6599109,Justin Brown,New Product,2024-02-18 04:17:13,2024-02-18 04:19:13,None,None,None,NaN
168224,6599109,Justin Brown,Cart Reminder,2024-02-19 04:17:13,2024-02-19 05:01:13,None,None,None,NaN


# Data preparation

In [9]:
data_emails.shape

(168226, 9)

In [10]:
data_emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168226 entries, 0 to 168225
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   customer_id         168226 non-null  int64  
 1   customer_name       168226 non-null  object 
 2   email_type          168226 non-null  object 
 3   sent_datetime       168226 non-null  object 
 4   open_datetime       96933 non-null   object 
 5   click_datetime      21726 non-null   object 
 6   bounce_datetime     3414 non-null    object 
 7   purchase_datetime   2290 non-null    object 
 8   transaction_amount  2290 non-null    float64
dtypes: float64(1), int64(1), object(7)
memory usage: 11.6+ MB


In [12]:
datetime_cols = [
    "sent_datetime",
    "open_datetime",
    "click_datetime",
    "bounce_datetime",
    "purchase_datetime"
]

for col in datetime_cols:
    data_emails[col] = pd.to_datetime(data_emails[col], errors="coerce") #convert to type 'datetime'



In [13]:
data_emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168226 entries, 0 to 168225
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   customer_id         168226 non-null  int64         
 1   customer_name       168226 non-null  object        
 2   email_type          168226 non-null  object        
 3   sent_datetime       168226 non-null  datetime64[ns]
 4   open_datetime       96933 non-null   datetime64[ns]
 5   click_datetime      21726 non-null   datetime64[ns]
 6   bounce_datetime     3414 non-null    datetime64[ns]
 7   purchase_datetime   2290 non-null    datetime64[ns]
 8   transaction_amount  2290 non-null    float64       
dtypes: datetime64[ns](5), float64(1), int64(1), object(2)
memory usage: 11.6+ MB


In [14]:
data_emails[["email_type", 
           "sent_datetime",
           "open_datetime", 
           "click_datetime", 
           "bounce_datetime",
           "purchase_datetime"]].groupby("email_type").count()

,sent_datetime,open_datetime,click_datetime,bounce_datetime,purchase_datetime
email_type,,,,,
Cart Reminder,33593,19314,7658,686,1350
New Product,33643,16063,2131,693,43
Promotion,33691,21120,3611,674,228
Reactivation,33549,15768,1411,661,93
Welcome,33750,24668,6915,700,576


In [15]:
#Create a binary flag showing whether a user interacted with an email

data_emails["sent_email"] = np.where(data_emails["sent_datetime"].notna(), 1, 0)
data_emails["opened_email"] = np.where(data_emails["open_datetime"].notna(), 1, 0)
data_emails["clicked_email"] = np.where(data_emails["click_datetime"].notna(), 1, 0)
data_emails["bounced_email"] = np.where(data_emails["bounce_datetime"].notna(), 1, 0)
data_emails["purchases"] = np.where(data_emails["purchase_datetime"].notna(), 1, 0)

In [16]:
data_emails.head()

,customer_id,customer_name,email_type,sent_datetime,open_datetime,click_datetime,bounce_datetime,purchase_datetime,transaction_amount,sent_email,opened_email,clicked_email,bounced_email,purchases
0,3023127,Zachary Stevenson,Welcome,2023-12-16 06:39:16,2023-12-16 06:44:16,NaT,NaT,NaT,NaN,1,1,0,0,0
1,3023127,Zachary Stevenson,Promotion,2023-12-17 06:39:16,NaT,NaT,NaT,NaT,NaN,1,0,0,0,0
2,3023127,Zachary Stevenson,New Product,2023-12-18 06:39:16,NaT,NaT,NaT,NaT,NaN,1,0,0,0,0
3,3023127,Zachary Stevenson,Cart Reminder,2023-12-19 06:39:16,2023-12-19 06:57:16,2023-12-19 07:11:16,NaT,NaT,NaN,1,1,1,0,0
4,3023127,Zachary Stevenson,Reactivation,2023-12-20 06:39:16,NaT,NaT,NaT,NaT,NaN,1,0,0,0,0


In [18]:
#Verify the number of interactions

data_emails[["email_type", 
           "sent_email",
           "opened_email", 
           "clicked_email", 
           "bounced_email",
           "purchases"]].groupby("email_type").sum()

,sent_email,opened_email,clicked_email,bounced_email,purchases
email_type,,,,,
Cart Reminder,33593,19314,7658,686,1350
New Product,33643,16063,2131,693,43
Promotion,33691,21120,3611,674,228
Reactivation,33549,15768,1411,661,93
Welcome,33750,24668,6915,700,576


In [26]:
data_emails.to_excel('data_email_compaign_23_24.xlsx', index=False)